In [1]:
# Libraries for feature extraction:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from tqdm import tqdm

# Libraries for pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [10]:
audio_dir = './Data/genres/'

In [11]:
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [12]:
def get_features(y, sr, n_fft = 1024, hop_length = 512):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None, 'zcr': None, 'chroma':None}
    
    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rmse'] = librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    
    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [19]:
def read_process_songs(src_dir, debug = True):    
    # Empty array of dicts with the processed features from all files
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
        
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)
                
                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))
                
                # Append the result to the data structure
                features = get_features(signal, sr)
                features['genre'] = genres[x]
                arr_features.append(features)
    return arr_features

In [20]:
# Get list of dicts with features and convert to dataframe
features = read_process_songs(audio_dir,debug=True)

Reading file: ./Data/genres/blues/blues.00088.wav
Reading file: ./Data/genres/blues/blues.00091.wav
Reading file: ./Data/genres/blues/blues.00023.wav
Reading file: ./Data/genres/blues/blues.00024.wav
Reading file: ./Data/genres/blues/blues.00070.wav
Reading file: ./Data/genres/blues/blues.00053.wav
Reading file: ./Data/genres/blues/blues.00085.wav
Reading file: ./Data/genres/blues/blues.00033.wav
Reading file: ./Data/genres/blues/blues.00019.wav
Reading file: ./Data/genres/blues/blues.00031.wav
Reading file: ./Data/genres/blues/blues.00020.wav
Reading file: ./Data/genres/blues/blues.00001.wav
Reading file: ./Data/genres/blues/blues.00046.wav
Reading file: ./Data/genres/blues/blues.00098.wav
Reading file: ./Data/genres/blues/blues.00017.wav
Reading file: ./Data/genres/blues/blues.00009.wav
Reading file: ./Data/genres/blues/blues.00040.wav
Reading file: ./Data/genres/blues/blues.00074.wav
Reading file: ./Data/genres/blues/blues.00086.wav
Reading file: ./Data/genres/blues/blues.00068.wav


In [21]:
df_features = pd.DataFrame(features)

In [31]:
df_features.to_csv('./Features/features.csv', index=False)

In [32]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
centroid_mean    1000 non-null float64
centroid_std     1000 non-null float64
chroma_mean      1000 non-null float64
chroma_std       1000 non-null float64
flux_mean        1000 non-null float64
flux_std         1000 non-null float64
genre            1000 non-null int64
mfcc_0_mean      1000 non-null float64
mfcc_0_std       1000 non-null float64
mfcc_10_mean     1000 non-null float64
mfcc_10_std      1000 non-null float64
mfcc_11_mean     1000 non-null float64
mfcc_11_std      1000 non-null float64
mfcc_12_mean     1000 non-null float64
mfcc_12_std      1000 non-null float64
mfcc_1_mean      1000 non-null float64
mfcc_1_std       1000 non-null float64
mfcc_2_mean      1000 non-null float64
mfcc_2_std       1000 non-null float64
mfcc_3_mean      1000 non-null float64
mfcc_3_std       1000 non-null float64
mfcc_4_mean      1000 non-null float64
mfcc_4_std       1000 non-null float64